Accuracy: 0.9
Demand at (12.9352, 77.6188) at 8:00: Low Demand
Demand at (12.9352, 77.6188) at 18:00: Low Demand


c:\Users\saiki\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\saiki\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [8]:
import pandas as pd

# Load dataset
df = pd.read_csv("Dataset/company_data.csv")

# Display first few rows
print(df.head())

# Check column names
print(df.columns)


                 Company Name  Latitude  Longitude
0             Infosys Limited   12.8452    77.6636
1               Wipro Limited   12.9063    77.6014
2              Biocon Limited   12.8380    77.6770
3  Bharat Electronics Limited   13.0399    77.5623
4   Tata Consultancy Services   12.9843    77.7118
Index(['Company Name', 'Latitude', 'Longitude'], dtype='object')


In [11]:
# Create synthetic demand data
high_demand = df.copy()
high_demand["Time_Minutes"] = np.random.randint(1020, 1140, len(df))  # Random times between 5-7 PM
high_demand["Demand"] = 1  # High Demand

low_demand = df.copy()
low_demand["Time_Minutes"] = np.random.choice(
    list(range(0, 1020)) + list(range(1140, 1440)), len(df)  # Times outside 5-7 PM
)
low_demand["Demand"] = 0  # Low Demand

# Combine both datasets
df_combined = pd.concat([high_demand, low_demand], ignore_index=True)

# Shuffle dataset
df_combined = df_combined.sample(frac=1, random_state=42).reset_index(drop=True)

# Save processed dataset
df_combined.to_csv("Dataset/company_data.csv", index=False)
print("Training data prepared successfully!")


Training data prepared successfully!


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

# Load the processed dataset
df = pd.read_csv("Dataset/company_data.csv")

# Define input features & target variable
X = df[["Latitude", "Longitude", "Time_Minutes"]]
y = df["Demand"]

# Split into training (80%) & testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
print("Model Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Save model
joblib.dump(model, "demand_prediction.pkl")
print("Model saved as demand_prediction.pkl")


Model Accuracy: 0.95
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.83      0.91         6
           1       0.93      1.00      0.97        14

    accuracy                           0.95        20
   macro avg       0.97      0.92      0.94        20
weighted avg       0.95      0.95      0.95        20

Model saved as demand_prediction.pkl


In [14]:
# Load the trained model
model = joblib.load("demand_prediction.pkl")

# Example test input
test_data = pd.DataFrame({
    "Latitude": [12.9716],  # Example location
    "Longitude": [77.5946],
    "Time_Minutes": [17 * 60 + 30]  # 5:30 PM
})

# Predict demand
prediction = model.predict(test_data)

# Output result
print("Predicted Demand:", "High" if prediction[0] == 1 else "Low")


Predicted Demand: High


In [15]:
# Load the trained model
model = joblib.load("demand_prediction.pkl")

# Example test input
test_data = pd.DataFrame({
    "Latitude": [12.9716],  # Example location
    "Longitude": [77.5946],
    "Time_Minutes": [7 * 60 + 30]  # 5:30 PM
})

# Predict demand
prediction = model.predict(test_data)

# Output result
print("Predicted Demand:", "High" if prediction[0] == 1 else "Low")


Predicted Demand: Low
